# Cayley-Dickson Construction Applied to Zi Definition

My original implementation of Gaussian integers included two classes, ``Zi`` and ``Qi``, where, for example, ``Zi(2, -7)`` represents a Gaussian integer, and ``Qi(-2/3, 4/5)`` represents a Gaussian rational.

I'd like to extend this code to include integer-valued quaternions and octonions. An elegant way to accomplish that goal would be to use the Cayley-Dickson construction, where complex numbers can be constructed from pairs of real numbers, quaternions can be constructed from pairs of those pairs, and octonions constructed from pairs of those pairs of pairs.

For more specifics, see my write-up about the Cayley-Dickson construction [at this link](https://abstract-algebra.readthedocs.io/en/latest/55_cayley_dickson.html).

In [1]:
# from fractions import Fraction
from random import randint

In [39]:
class Zi:
    """Pairs of integers (Gaussian Integers), Pairs of pairs of integers (Quaternion Integers),
    Pairs of pairs of pairs of integers (Octonion Integers), etc."""
    
    def __init__(self, re=None, im=None):

        if isinstance(re, (float, int)):
            self.__re = round(re)
            if im is None:
                self.__im = 0
            elif isinstance(im, (float, int)):
                self.__im = round(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")
        elif isinstance(re, complex):
            if im is None:
                self.__re = round(re.real)
                self.__im = round(re.imag)
            elif isinstance(im, (complex, Zi)):
                self.__re = Zi(re)
                self.__im = Zi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")
        elif isinstance(re, Zi):
            if im is None:
                self.__re = re.real
                self.__im = re.imag
            elif isinstance(im, (complex, Zi)):
                self.__re = Zi(re)
                self.__im = Zi(im)
            else:
                raise Exception(f"Inputs incompatible: {re} and {im}")
        elif re is None:
            if im is None:
                self.__re = 0
                self.__im = 0
            else:
                raise Exception("If re is None, then im must be None. But im = {im}")
        else:
            raise Exception("We should never get to this point in the code")
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.__re}, {self.__im})"
                                
    @property
    def real(self):
        return self.__re
    
    @property
    def imag(self):
        return self.__im

    def __neg__(self):
        return Zi(- self.__re, - self.__im)

    def __add__(self, other):
        return Zi(self.__re + other.real, self.__im + other.imag)

    def __sub__(self, other):
        return Zi(self.__re - other.real, self.__im - other.imag)

    # Cayley-Dickson Construction
    # See [Schafer, 1966]
    # Conjugation (*) define recursively: a* = a and (u, v)* = (u*, -v)
    # Multiplication: (a, b) x (c, d) = (a x c  +  mu x d x b*,  a* x d  +  c x b)
    # where mu is hardcoded below to be -1, for now.
    def __mul__(self, other):
        a, b, c, d = self.__re, self.__im, other.real, other.imag
        real_part = a * c - d * b.conjugate()
        imag_part = a.conjugate() * d + c * b
        return Zi(real_part, imag_part)

    def __rmul__(self, other):
        return self * other

    def conjugate(self):
        return Zi(self.__re, - self.__im)

    def norm(self):
        if isinstance(self.__re, (float, int)) and isinstance(self.__im, (float, int)):
            n = self * self.conjugate()
            return n.real
        elif isinstance(self.__re, Zi) and isinstance(self.__im, Zi):
            p = self.__re.norm() + self.__im.norm()
            return p
        else:
            raise Exception(f"Can't take norm of {self}")
            
    def is_base(self):
        """Return True if this Zi is a base Zi, e.g., the re & im parts
        are numbers, not other Zis"""
        return isinstance(self.__re, (float, int)) and isinstance(self.__im, (float, int))

    @staticmethod
    def random(re1=-100, re2=100, im1=-100, im2=100):
        """Return a random Zi(re, im) with integer components where,
        re1 <= re <= re2 and im1 <= im <= im2."""
        return Zi(randint(re1, re2), randint(im1, im2))

    # A convenience function for testing purposes:
    def make_complex(self):
        if isinstance(self.__re, (float, int)) and isinstance(self.__im, (float, int)):
            return complex(self.__re, self.__im)
        elif isinstance(self.__re, Zi) and isinstance(self.__im, Zi):
            return Zi(self.make_complex(self.__re), self.make_complex(self.__im))
        else:
            raise Exception(f"Can't make complex out of {self}")

    # --------------------------

    def __radd__(self, other):
        """The reflected (swapped) operand for addition: other + self"""
        return Zi(other) + self

    def __iadd__(self, other):
        """Implements the += operation: self += other"""
        return Zi(self.real + other.real, self.imag + other.imag)

    def __rsub__(self, other):
        """The reflected (swapped) operand for subtraction: other - self"""
        return Zi(other) - self

    def __isub__(self, other):
        """Implements the -= operation: self -= other"""
        return Zi(self.real - other.real, self.imag - other.imag)

    def __imul__(self, other):
        """Implements the *= operation: self *= other"""
        a = self.real
        b = self.imag
        c = round(other.real)
        d = round(other.imag)
        return Zi(a * c - b * d, a * d + b * c)

    def __pow__(self, n: int, modulo=None):
        """Implements the ** operator: self ** n.

        If n == 0, then Zi(1, 0) is returned. If n < 0, then the Gaussian
        rational, Qi, for 1 / self**n is returned. Otherwise, self ** n is returned.
        """
        result = self
        if isinstance(n, int):
            if n == 0:
                result = Zi(1)  # "1"
            elif n > 0:
                for _ in range(n - 1):
                    result = result * self
            else:  # n < 0
                result = 1 / (self ** abs(n))
        else:
            raise TypeError(f"The power, {n}, must be an integer.")
        return result

    def __eq__(self, other: Zi) -> bool:
        """Return True if this Zi equals other."""
        return (self.real == other.real) and (self.imag == other.imag)

    def __ne__(self, other) -> bool:
        """Return True if this Zi does NOT equal other."""
        return (self.real != other.real) or (self.imag != other.imag)

    def __hash__(self):
        """Allow this Zi to be hashed."""
        return hash((self.real, self.imag))

    def __abs__(self) -> float:
        """Returns the square root of the norm."""
        return math.sqrt(self.norm)

    def __pos__(self):
        return +self

    def __rpow__(self, base):
        return NotImplemented

    @staticmethod
    def eye():
        """Return i = Zi(0, 1)"""
        return Zi(0, 1)

    @staticmethod
    def units():
        """Returns the list of four units, [1, -1, i, -i], as Zis."""
        return [Zi(1), -Zi(1), Zi.eye(), -Zi.eye()]

    @property
    def is_unit(self):
        """Returns True if this Zi is a unit."""
        return self in Zi.units()

    @staticmethod
    def two():
        """Returns 1+i, because a Gaussian integer has an even norm if and only if
        it is a multiple of 1+i."""
        return Zi(1, 1)

    def associates(self):
        """Return a list of this Zi's three associates"""
        us = Zi.units()
        return list(map(lambda u: u * self, us[1:]))  # skip multiplying by 1

    def is_associate(self, other):
        """Return True if the other Zi is an associate of this Zi

        Otherwise, return False.
        """
        q = self // other
        if q:
            if q in Zi.units():
                return True
            else:
                return False
        else:
            return FalseL


In [40]:
Zi()

Zi(0, 0)

In [41]:
Zi(1)

Zi(1, 0)

In [42]:
Zi(1, 2)

Zi(1, 2)

In [43]:
Zi(1.9, 2.1)

Zi(2, 2)

In [44]:
Zi((1.9+2.1j))

Zi(2, 2)

In [45]:
Zi((1+2j), (3+4j))

Zi(Zi(1, 2), Zi(3, 4))

In [46]:
Zi(Zi(1, 2))

Zi(1, 2)

In [47]:
Zi(Zi(), Zi(1))

Zi(Zi(0, 0), Zi(1, 0))

In [48]:
Zi(Zi(3, 4), (1+2j))

Zi(Zi(3, 4), Zi(1, 2))

In [49]:
Zi(Zi(Zi(0), Zi(1)), Zi(Zi(3, 4), Zi(1, 2)))

Zi(Zi(Zi(0, 0), Zi(1, 0)), Zi(Zi(3, 4), Zi(1, 2)))

In [50]:
Zi(Zi(Zi(0), Zi(1)), Zi(Zi(3, 4), (1+2j)))

Zi(Zi(Zi(0, 0), Zi(1, 0)), Zi(Zi(3, 4), Zi(1, 2)))

In [51]:
def random_quaternion(size=10):
    a = -size; b = size
    return Zi(Zi.random(a, b, a, b), Zi.random(a, b, a, b))

def random_octonion(size=10):
    return Zi(random_quaternion(size), random_quaternion(size))

## Examples

In [52]:
n = 4
zs = [Zi.random(-10, 10, -10, 10) for i in range(n)]
z1 = zs[0]
z2 = zs[1]
z3 = zs[2]
z4 = zs[3]
print(f"{zs = }")
print(f"{z1 = }")
print(f"{z2 = }")
print(f"{z3 = }")
print(f"{z4 = }")

zs = [Zi(-1, 2), Zi(-6, -4), Zi(8, 2), Zi(-4, 0)]
z1 = Zi(-1, 2)
z2 = Zi(-6, -4)
z3 = Zi(8, 2)
z4 = Zi(-4, 0)


In [53]:
print(f"{z1 = }")
print(f"{-z1 = }")
print(f"{z1.real = }")
print(f"{z1.imag = }")
print(f"{z1.conjugate() = }")
print(f"{z1.norm() = }")
print(f"{z1.is_base() = }")

z1 = Zi(-1, 2)
-z1 = Zi(1, -2)
z1.real = -1
z1.imag = 2
z1.conjugate() = Zi(-1, -2)
z1.norm() = 5
z1.is_base() = True


In [54]:
print(f"{z1 = }")
print(f"{z2 = }")
print(f"{z1 + z2 = }")
print(f"{z1 - z2 = }")

z1 = Zi(-1, 2)
z2 = Zi(-6, -4)
z1 + z2 = Zi(-7, -2)
z1 - z2 = Zi(5, 6)


For comparisons, create complex numbers corresponding to z1, z2, z3, and z4

In [55]:
c1 = z1.make_complex()
c2 = z2.make_complex()
c3 = z3.make_complex()
c4 = z4.make_complex()
# print(z1, z2, z3, z4)
# print(c1, c2, c3, c4)
print(f"{z1 = } --> {c1 = }")
print(f"{z2 = } --> {c2 = }")
print(f"{z3 = } --> {c3 = }")
print(f"{z4 = } --> {c4 = }")

z1 = Zi(-1, 2) --> c1 = (-1+2j)
z2 = Zi(-6, -4) --> c2 = (-6-4j)
z3 = Zi(8, 2) --> c3 = (8+2j)
z4 = Zi(-4, 0) --> c4 = (-4+0j)


In [56]:
print(f"{z1 * z2 = }")
print(f"{z1 * 2 = }")
print(f"{2 * z1 = }")
print(f"{c1 * c2 = }")

z1 * z2 = Zi(14, -8)
z1 * 2 = Zi(-2, 4)
2 * z1 = Zi(-2, 4)
c1 * c2 = (14-8j)


In [57]:
q1 = Zi(z1, z2)
q2 = Zi(z3, z4)
d1 = Zi(c1, c2)
d2 = Zi(c3, c4)

print(f"{q1.is_base() = }")
print(f"{q1 = }")
print(f"{q2 = }\n")

print(f"{d1 = }")
print(f"{d2 = }\n")

print(f"{q1 + q2 = }")
print(f"{q1 * 2 = }")
print(f"{2 * q1 = }")
print(f"{q1 * q2 = }\n")

print(f"{d1 + d2 = }")
print(f"{d1 * d2 = }")

q1.is_base() = False
q1 = Zi(Zi(-1, 2), Zi(-6, -4))
q2 = Zi(Zi(8, 2), Zi(-4, 0))

d1 = Zi(Zi(-1, 2), Zi(-6, -4))
d2 = Zi(Zi(8, 2), Zi(-4, 0))

q1 + q2 = Zi(Zi(7, 4), Zi(-10, -4))
q1 * 2 = Zi(Zi(-2, 4), Zi(-12, -8))
2 * q1 = Zi(Zi(-2, 4), Zi(-12, -8))
q1 * q2 = Zi(Zi(-36, 30), Zi(-36, -36))

d1 + d2 = Zi(Zi(7, 4), Zi(-10, -4))
d1 * d2 = Zi(Zi(-36, 30), Zi(-36, -36))


In [58]:
o1 = Zi(q1, q2)
print(f"{o1 = }")

o1 = Zi(Zi(Zi(-1, 2), Zi(-6, -4)), Zi(Zi(8, 2), Zi(-4, 0)))


In [59]:
o1.norm()

141

In [60]:
p1.norm() + p2.norm() + p3.norm() + p4.norm()

273

In [61]:
q1.norm()

57

In [62]:
qx = random_quaternion()
qy = random_quaternion()

In [63]:
qx

Zi(Zi(-9, -10), Zi(-4, -1))

In [64]:
qy

Zi(Zi(5, 3), Zi(6, -10))

In [65]:
foo = qx * qx.conjugate()
foo

Zi(Zi(-2, 180), Zi(-20, 80))

In [66]:
foo.norm()

39204